In [1]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall git+https://github.com/The-AI-Alliance/gofannon.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.9 MB/s eta 0:

In [16]:
from openai import OpenAI
from google.colab import userdata
import json

from gofannon.reasoning import SequentialCoT
from gofannon.basic_math.complex import complex_response

scot_depth_chart = [
    {'model_name' : "Qwen/Qwen2.5-72B-Instruct",
     'base_url' : "https://api.deepinfra.com/v1/openai",
     'api_key' : userdata.get('deepinfra'),
     'temperature' : 0.3,
     'prompt_appendix' : 'This isnt used at level 0.'
     },
    {'model_name' : "meta-llama/Llama-3.3-70B-Instruct",
     'base_url' : "https://api.deepinfra.com/v1/openai",
     'api_key' : userdata.get('deepinfra'),
     'temperature' : 0.3,
     'prompt_appendix' : 'This isnt used at level 1.'
     },
    {'model_name' : "meta-llama/Llama-3.3-70B-Instruct",
     'base_url' : "https://api.deepinfra.com/v1/openai",
     'api_key' : userdata.get('deepinfra'),
     'temperature' : 0.3,
     'prompt_appendix' : """Compile all of the previous information into one cohesive document.

     Do not summarize or otherwise truncate the content, except in the case of formatting.

     Your Response should look like a report to an executive."""
     }
]

# If you want to use a different reasoning model, you set it here.
tool_list = [F(depth_chart=scot_depth_chart, steps= 4) for F in [SequentialCoT]]
tool_map = {f.name: f.fn for f in tool_list}
tool_desc_list = [f.definition for f in tool_list]

In [17]:
prompt = """
Give me a step-by-step plan for building an AI Lab.
"""
# here is the user request
messages = [
    {
        "role": "user",
        "content": prompt
    }
]

openai = OpenAI(
    api_key=userdata.get('deepinfra'),
    base_url="https://api.deepinfra.com/v1/openai",
)
# let's send the request and print the response
response = openai.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct", # Model at this level is just deciding what tool to call.
    messages=messages,
    tools=tool_desc_list,
    tool_choice="auto",
)
tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    print(tool_call.model_dump())


{'id': 'call_ZELlA8Y0R6FcPvGc9Xot5ZAz', 'function': {'arguments': '{"prompt": "Building an AI Lab"}', 'name': 'sequential_cot'}, 'type': 'function'}


In [18]:
function_responses = []
for tool_call in tool_calls:
  function_name = tool_call.function.name
  function_args = json.loads(tool_call.function.arguments)
  function_response = tool_map[function_name](**function_args)
  function_responses.append(function_response)
  # extend conversation with function response
  messages.append({
      "tool_call_id": tool_call.id,
      "role": "tool",
      "content": function_response,
  })

print("="*80)
print(function_response.choices[0].message.content)

thinking about Step 1/4:'Research and identify the key components and technologies needed for an AI lab, including hardware, software, and tools.'...
thinking about Step 2/4:'Determine the specific goals and use cases for the AI lab to tailor the setup and resource allocation.'...
thinking about Step 3/4:'Outline a budget and resource plan, including costs for equipment, software licenses, and ongoing maintenance.'...
thinking about Step 4/4:'Develop a timeline and implementation plan, including milestones for setup, testing, and deployment of the AI lab.'...
thinking about how it all fits together
**Building an AI Lab: A Comprehensive Report**

**Executive Summary**

This report provides a detailed overview of the key components, technologies, and steps required to build an AI lab. The report covers the research and identification of key components and technologies, determination of specific goals and use cases, budget and resource planning, and development of a timeline and implement